<link rel='stylesheet' href='../assets/css/main.css'/>

[<< back to main index](../README.md)

# Multiple Linear Regression Lab : House Prices

### Overview
Estimate house prices using MLR

### Builds on
None

### Run time
approx. 20 minutes

### Notes



In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print('Spark UI running on http://YOURIPADDRESS:' + sc.uiWebUrl.split(':')[2])

Spark UI running on http://YOURIPADDRESS:4040



## Step 1 : House data

In [2]:
%%time

housePrices = spark.read.csv("/data/house-prices/house-sales-full.csv", \
                             header=True, inferSchema=True)
print("read {:,} records".format(housePrices.count()))


read 27,063 records
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 8.69 s


In [3]:
housePrices.printSchema()
housePrices.show()

root
 |-- DocumentID: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- SalePrice: integer (nullable = true)
 |-- PropertyID: long (nullable = true)
 |-- PropertyType: string (nullable = true)
 |-- ym: string (nullable = true)
 |-- zhvi_px: integer (nullable = true)
 |-- zhvi_idx: double (nullable = true)
 |-- AdjSalePrice: decimal(8,0) (nullable = true)
 |-- NbrLivingUnits: integer (nullable = true)
 |-- SqFtLot: integer (nullable = true)
 |-- SqFtTotLiving: integer (nullable = true)
 |-- SqFtFinBasement: integer (nullable = true)
 |-- Bathrooms: double (nullable = true)
 |-- Bedrooms: integer (nullable = true)
 |-- BldgGrade: integer (nullable = true)
 |-- YrBuilt: integer (nullable = true)
 |-- YrRenovated: integer (nullable = true)
 |-- TrafficNoise: integer (nullable = true)
 |-- LandVal: integer (nullable = true)
 |-- ImpsVal: integer (nullable = true)
 |-- ZipCode: integer (nullable = true)
 |-- NewConstruction: boolean (nullable = true)

+----------+--------+--

Wow! That's a lot of columns.  Maybe we should foucs on just a few of them.

**=> Select only "SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot" **

In [4]:
# TODO: select only "SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"
housePrices_small = housePrices.select("SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot")
housePrices_small.show()

+---------+--------+---------+-------------+-------+
|SalePrice|Bedrooms|Bathrooms|SqFtTotLiving|SqFtLot|
+---------+--------+---------+-------------+-------+
|   280000|       6|      3.0|         2400|   9373|
|  1000000|       4|     3.75|         3764|  20156|
|   745000|       4|     1.75|         2060|  26036|
|   425000|       5|     3.75|         3200|   8618|
|   240000|       4|     1.75|         1720|   8620|
|   349900|       2|      1.5|          930|   1012|
|   327500|       3|      1.5|         1750|  34465|
|   347000|       4|     1.75|         1860|  14659|
|   220400|       2|      1.0|          990|   5324|
|   437500|       4|      2.0|         1980|  10585|
|   150000|       2|      1.0|          840|  12750|
|   300000|       3|      1.0|         1750|   5200|
|   292000|       2|      1.0|          790|   5240|
|   299800|       2|      1.0|          790|   5240|
|   370000|       3|     2.25|         1530|   1245|
|   605000|       3|     2.75|         2120|  

## Step 2: Create Features and Label Column in Dataframe

We will create our "features" column, which is of type vector, and our label column, for which we can just reuse the SquareFootage column.





In [5]:
from pyspark.ml.feature import VectorAssembler

## TODO : Build features with  "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"
## Hint : inputCols = ['Bedrooms', 'Bathrooms', ??? , ??? ]

assembler = VectorAssembler(inputCols=["Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"], outputCol="features")
featureVector = assembler.transform(housePrices_small)

## TODO : create a label column to mirror 'SalePrice'
featureVector = featureVector.withColumn("label", featureVector["SalePrice"])
featureVector.show(20, False)

+---------+--------+---------+-------------+-------+-------------------------+-------+
|SalePrice|Bedrooms|Bathrooms|SqFtTotLiving|SqFtLot|features                 |label  |
+---------+--------+---------+-------------+-------+-------------------------+-------+
|280000   |6       |3.0      |2400         |9373   |[6.0,3.0,2400.0,9373.0]  |280000 |
|1000000  |4       |3.75     |3764         |20156  |[4.0,3.75,3764.0,20156.0]|1000000|
|745000   |4       |1.75     |2060         |26036  |[4.0,1.75,2060.0,26036.0]|745000 |
|425000   |5       |3.75     |3200         |8618   |[5.0,3.75,3200.0,8618.0] |425000 |
|240000   |4       |1.75     |1720         |8620   |[4.0,1.75,1720.0,8620.0] |240000 |
|349900   |2       |1.5      |930          |1012   |[2.0,1.5,930.0,1012.0]   |349900 |
|327500   |3       |1.5      |1750         |34465  |[3.0,1.5,1750.0,34465.0] |327500 |
|347000   |4       |1.75     |1860         |14659  |[4.0,1.75,1860.0,14659.0]|347000 |
|220400   |2       |1.0      |990          

##  Step 3: Run the Linear Regression Model


In [6]:
from pyspark.ml.regression import LinearRegression

## Hint : Run model with max iterations as 10
#lr = LinearRegression(maxIter=???)
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(featureVector)

print("Coefficents:" + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))



Coefficents:[-69405.45781195552,25714.1481078027,274.45831276947166,-0.0]
Intercept: 105562.58117252712


**Insepct the coeffiients.  Are you seeing any ZERO (0) values?  What is the meaning of this?**

In [11]:
print("numIterations: %d" % lrModel.summary.totalIterations)
print("objectiveHistory: %s" % str(lrModel.summary.objectiveHistory))
# lrModel.summary.residuals.show()
print("RMSE: %f" % lrModel.summary.rootMeanSquaredError)

## TODO : print 'r2' from training summary
## Hint : Use TAB completion
print("r2: %f" % lrModel.summary.r2)


numIterations: 11
objectiveHistory: [0.5000000000000002, 0.4565457266170319, 0.3376986997173243, 0.30010335871879656, 0.2852977093323423, 0.27654782886180707, 0.2701349132960898, 0.26332624819112604, 0.26138384191458414, 0.25967594157902535, 0.25839392000729794]
RMSE: 246442.225880
r2: 0.483214


#### Observe the summary output
- How many iterations did it do?
- Is it more than our 'max' iterations?
- If so, increase the 'maxIter' to 50 and run it again
- How many iterations is done now?  
- Can you explain the behavior?
- Also did the coefficients change from  maxIter=10  to maxIter=50 ?
- Did r2 change?  Why / Why-not?



## Step 3:  Predict some home prices
Create a new data frame with the following data

In [12]:

# Make a Pandas dataframe
new_data = pd.DataFrame({'Bedrooms' : [5,3,2],
                         'Bathrooms' : [3,2,1.5],
                         'SqFtTotLiving' : [4400, 1800, 1550],
                         'SqFtLot' : [10000, 5000, 4000]
             })

new_data_spark = spark.createDataFrame(new_data)
new_featureVector = assembler.transform(new_data_spark)  #run VectorAssembler again to create features.
new_featureVector.show()




+--------+---------+-------------+-------+--------------------+
|Bedrooms|Bathrooms|SqFtTotLiving|SqFtLot|            features|
+--------+---------+-------------+-------+--------------------+
|       5|      3.0|         4400|  10000|[5.0,3.0,4400.0,1...|
|       3|      2.0|         1800|   5000|[3.0,2.0,1800.0,5...|
|       2|      1.5|         1550|   4000|[2.0,1.5,1550.0,4...|
+--------+---------+-------------+-------+--------------------+



## Step 4: Run on some prediction data

Lets see the predicted output.

In [13]:
## TODO: transform dataframe with new feature vector to get predictions
## Hint : new_featureVector

predicted_prices = lrModel.transform(new_featureVector)
predicted_prices.show()

+--------+---------+-------------+-------+--------------------+------------------+
|Bedrooms|Bathrooms|SqFtTotLiving|SqFtLot|            features|        prediction|
+--------+---------+-------------+-------+--------------------+------------------+
|       5|      3.0|         4400|  10000|[5.0,3.0,4400.0,1...|1043294.3126218329|
|       3|      2.0|         1800|   5000|[3.0,2.0,1800.0,5...|442799.46693731495|
|       2|      1.5|         1550|   4000|[2.0,1.5,1550.0,4...| 430733.2725030012|
+--------+---------+-------------+-------+--------------------+------------------+



## Step 5 : Some more FUN
Now let's feed 'SalePrice' as an input to our VectorAssembler.  
In Step-2, try this

```python
assembler = VectorAssembler(inputCols=["SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"],
                            outputCol="features")
```

Run again, and observe the following:
- look at the coefficients  (why are most of them zero?)
- look at r2

Can you explain what is going on :-) 